In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)
import pandas as pd
from src.model import TreeData
from src.config import PATH_PROCESSED
import json

import requests
import json
from src.model import TreeData
from src.config import PATH_PROCESSED
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import numpy as np

/workspaces/street_tree


2025-03-28 03:16:35,343 - src.utils - INFO - Logger is ready
2025-03-28 03:16:35,345 - src.utils - INFO - This is a test log message.
2025-03-28 03:16:35,347 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-28 03:16:35,347 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-28 03:16:35,349 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-28 03:16:35,350 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-28 03:16:35,351 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-28 03:16:35,351 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-28 03:16:35,353 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-28 03:16:35,354 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-28 03:16:35,355 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-28 03:16:35,356 - src.utils - INFO - output_path: /workspaces/street_t

In [2]:
# Пример использования:
data1 = pd.read_csv(f"{PATH_PROCESSED}/test.csv")
df = data1.drop('health', axis=1)
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
data_json = df.to_dict(orient='records')
json_data = json.dumps(data_json)

tree_data_list = TreeData.from_dataframe(json_data)

# Convert list of TreeData objects to list of dictionaries
data = [tree_data.dict() for tree_data in tree_data_list]

# Convert list of dictionaries to JSON
json_data = json.dumps(data)


# Send POST request to the endpoint
url = "http://localhost:8000/predict_health/"
headers = {"Content-Type": "application/json"}
response = requests.post(url, data=json_data, headers=headers)
print(response) # <Response [200]>

/tmp/ipykernel_66786/2633686050.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data = [tree_data.dict() for tree_data in tree_data_list]


<Response [200]>


In [3]:
print(json_data[:1000])

[{"tree_id": 536325, "block_id": 415591, "tree_dbh": 56, "curb_loc": 0, "spc_latin": "Fraxinus", "steward": "None", "guards": "Unsure", "sidewalk": 0, "user_type": "NYC Parks Staff", "problems": "NoProblem", "root_stone": 0, "root_grate": 0, "root_other": 0, "trunk_wire": 0, "trnk_light": 0, "trnk_other": 0, "brch_light": 0, "brch_shoe": 0, "brch_other": 0, "postcode": 10306, "borough": "Staten Island", "cncldist": 50, "st_assem": 62, "st_senate": 24, "nta": "SI24", "boro_ct": 5027900, "latitude": 40.571808, "longitude": -74.14425163}, {"tree_id": 247341, "block_id": 335158, "tree_dbh": 16, "curb_loc": 1, "spc_latin": "Acer platanoides", "steward": "None", "guards": "Unsure", "sidewalk": 0, "user_type": "NYC Parks Staff", "problems": "RootOther,TrunkOther,BranchOther", "root_stone": 0, "root_grate": 0, "root_other": 1, "trunk_wire": 0, "trnk_light": 0, "trnk_other": 1, "brch_light": 0, "brch_shoe": 0, "brch_other": 1, "postcode": 11426, "borough": "Queens", "cncldist": 23, "st_assem": 

In [4]:
from sklearn.metrics import roc_auc_score
import numpy as np

# Истинные метки
true_labels = data1['health'].map({'Poor': 0, 'Fair': 1, 'Good': 2}).tolist()

# Извлекаем вероятности из ответа API
predicted_probabilities = np.array([item['probably'] for item in response.json()])

# Рассчитываем AUC-ROC
roc_auc = roc_auc_score(true_labels, predicted_probabilities, multi_class='ovr')

# Выводим результат
print(f"AUC-ROC: {roc_auc}")

AUC-ROC: 0.7031976956450178
